In [ ]:
# Z-score Analysis for Calcium Imaging Data
# Z-score = (F - median(F)) / MAD(F), where MAD is the median absolute deviation
# Response classification based on statistical thresholds:
# - If max_peak > mean + 2*std: Cell Excited
# - If max_peak < mean - 2*std: Cell Inhibited
# - Otherwise: Cell Not Responsive
# Save Z-score data and response classification to CSV files

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os

# Define the file path
filename = "path/to/your/file.csv"  # Replace with your actual file path

# Load the CSV file
try:
    data = pd.read_csv(filename)
except (FileNotFoundError, pd.errors.EmptyDataError, pd.errors.ParserError) as e:
    print(f"Error loading file: {e}")
    exit()

# Identify cell columns (columns that start with 'cell-')
cell_columns = [col for col in data.columns if col.startswith('cell-')]

# Extract time and fluorescence data
time = data.iloc[:, 0]  # First column is assumed to be time
F = data[cell_columns]  # Remaining columns are fluorescence data for each cell

# Recalibrate time to align with a specific event (e.g., Entry time in Novel Context)
specific_time = 100  # Replace with the actual time of the specific event
recalibrated_time = time - specific_time

# Define the time window of interest (-60s to 120s)
time_mask = (recalibrated_time >= -60) & (recalibrated_time <= 120)
time_filtered = recalibrated_time[time_mask]
F_filtered = F.loc[time_mask, cell_columns]  # Extract fluorescence data for the time window

def median_zscore(F_cell, baseline_mask):
    """
    Calculate the median-based Z-score using ONLY the baseline period (-20s to 0s).
    Z-score = (F_cell - median(F_baseline)) / MAD(F_baseline).
    """
    F_baseline = F_cell[baseline_mask]
    median_F = np.median(F_baseline)
    mad_F = np.median(np.abs(F_baseline - median_F))
    return (F_cell - median_F) / mad_F if mad_F != 0 else np.zeros_like(F_cell)

def determine_response(mean_val, std_val, max_peak):
    """
    Determine the cell response type based on statistical thresholds.
    - If max_peak > mean + 2*std: Cell Excited
    - If max_peak < mean - 2*std: Cell Inhibited
    - Otherwise: Cell Not Responsive
    """
    if max_peak > (mean_val + 2 * std_val):
        return "Cell Excited"
    elif max_peak < (mean_val - 2 * std_val):
        return "Cell Inhibited"
    else:
        return "Cell Not Responsive"

def plot_zscore_signal(time, z_score, cell_name, mean_val, std_val, max_peak, response):
    """
    Plot the Z-score signal for a cell with annotations.
    """
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=time, y=z_score, mode='lines', name=f'Z-score ({cell_name})', line=dict(color='red', width=1)))
    
    # Add vertical line at t=0 to mark Entry Time
    fig.add_shape(type="line", x0=0, x1=0, y0=min(z_score), y1=max(z_score), line=dict(color="red", width=2, dash="dot"))
    
    # Add annotations with statistics and response type
    annotation_text = f"Mean: {mean_val:.2f}\nSTD: {std_val:.2f}\nMax Peak: {max_peak:.2f}\n{response}"
    fig.add_annotation(x=50, y=max(z_score), text=annotation_text, showarrow=False, font=dict(size=12))
    
    # Update layout
    fig.update_layout(title=f"Z-score Signal for {cell_name}", xaxis_title='Time (s)', yaxis_title='Median-based Z-score', legend_title='Signal')
    fig.show()

# Dictionary to store Z-scores and response classifications
Z_dict = {}
response_dict = {}

# Define masks for baseline (-60s to 0s) and peak (0s to 60s) periods
baseline_mask = (time_filtered >= -60) & (time_filtered < 0)
peak_mask = (time_filtered >= 0) & (time_filtered <= 60)

# Process each cell
for cell in cell_columns:
    F_cell = F_filtered[cell]
    Z = median_zscore(F_cell, baseline_mask)  # Z-score calculated AFTER masking
    
    # Store Z-score values
    Z_dict[cell] = Z
    
    # Calculate statistics for baseline and peak
    mean_baseline = np.mean(Z[baseline_mask])
    std_baseline = np.std(Z[baseline_mask])
    max_peak = np.max(Z[peak_mask])
    
    # Determine response type
    response = determine_response(mean_baseline, std_baseline, max_peak)
    response_dict[cell] = response
    
    # Plot Z-score signal
    plot_zscore_signal(time_filtered, Z, cell, mean_baseline, std_baseline, max_peak, response)

# Save Z-score data to a CSV file
output_dir = "directory/to/save/file.csv"  # Replace with your desired output directory
output_filename = 'name_of_the_file.csv'  # Replace with your desired output filename
output_filepath = os.path.join(output_dir, output_filename)

df_final = pd.DataFrame(Z_dict)
df_final.insert(0, 'time', time_filtered)  # Insert filtered time column
df_final.to_csv(output_filepath, index=False)

# Save response classification to a CSV file
response_filename = 'name_of_the_file.csv'  # Replace with your desired output filename
response_filepath = os.path.join(output_dir, response_filename)
pd.DataFrame.from_dict(response_dict, orient='index', columns=['Response']).to_csv(response_filepath)

print(f"Z-score data saved to {output_filepath}")
print(f"Response classification saved to {response_filepath}")